In [ ]:
import base64
import json
import requests

API_KEY = "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"
AUDIO_PATH = "hkt_wargaming_audio.mp3"

OUTPUT_TRANSCRIPT_PATH = "transkript_mit_sprechern.txt"
OUTPUT_ONEPAGER_JSON_PATH = "one_pager.json"

MODEL_TRANSCRIBE = "openai/gpt-4o-audio-preview"
MODEL_SUMMARY = "openai/gpt-4.1-mini"
BASE_URL = "https://openrouter.ai/api/v1/chat/completions"


def transcribe_audio():
    with open(AUDIO_PATH, "rb") as f:
        audio_b64 = base64.b64encode(f.read()).decode("utf-8")

    payload = {
        "model": MODEL_TRANSCRIBE,
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": (
                            "Transkribiere dieses Audio vollständig auf Deutsch. "
                            "Erkenne Sprecher automatisch und nummeriere sie "
                            "('Sprecher 1:', 'Sprecher 2:' etc.). "
                            "Jeder Sprecherwechsel beginnt mit neuer Zeile. "
                            "Gib NUR das Transkript aus."
                        ),
                    },
                    {
                        "type": "input_audio",
                        "input_audio": {"data": audio_b64, "format": "mp3"},
                    },
                ],
            }
        ],
    }

    headers = {"Authorization": f"Bearer {API_KEY}", "Content-Type": "application/json"}
    resp = requests.post(BASE_URL, headers=headers, json=payload)
    resp.raise_for_status()

    text = resp.json()["choices"][0]["message"]["content"]

    with open(OUTPUT_TRANSCRIPT_PATH, "w", encoding="utf-8") as f:
        f.write(text)

    return text


def create_one_pager_json(transcript: str):
    template = """
Erstelle aus folgendem Transkript einen kurzen One-Pager mit den Kernaussagen der Diskussion.
Nutze nur Informationen aus dem Transkript. Wenn etwas nicht eindeutig ist, formuliere neutral.

Gib das Ergebnis als reines JSON mit GENAU diesem Schema zurück (ohne zusätzliche Felder):

{
  "runde1": {
    "phase1": {
      "intel_blau": "Blau hat die 12 nicht erreicht und daher ist der Intelvorteil von Rot geheim.",
      "intel_rot": "Rot hat keinen Intelvorteil, da 1 gewürfelt."
    },
    "phase2": {
      "ressourcen": "S (1) eine kurze Zusammenfassung der Argumentation zu den verfügbaren Ressourcen.",
      "komplexitaet": "M (2) eine kurze Zusammenfassung der Argumentation zur Komplexität.",
      "verteidigung": "L (3) eine kurze Zusammenfassung der Argumentation zur Verteidigung.",
      "angriffszeitfenster": "M (2) eine kurze Zusammenfassung der Argumentation zum Angriffszeitfenster.",
      "tabellensumme": 8,
      "erfolgswurf_rot": "erfolgreich",
      "auswirkungen": "Voller Erfolg – eine knappe Beschreibung der operativen Auswirkungen gemäß Zusammenfassung des Spielleiters."
    }
  },
  "runde2": {
    "phase1": {
      "intel_blau": "Blau hat die 12 nicht erreicht und daher ist der Intelvorteil von Rot geheim.",
      "intel_rot": "Rot hat einen Intelvorteil von +2 (da 5 gewürfelt) auf Angriffe gegen Personal (da 4 gewürfelt)."
    },
    "phase2": {
      "ressourcen": "M (2) eine kurze Zusammenfassung der Argumentation zu den verfügbaren Ressourcen.",
      "komplexitaet": "L (3) eine kurze Zusammenfassung der Argumentation zur Komplexität.",
      "verteidigung": "M (2) eine kurze Zusammenfassung der Argumentation zur Verteidigung.",
      "angriffszeitfenster": "S (1) eine kurze Zusammenfassung der Argumentation zum Angriffszeitfenster.",
      "tabellensumme": 8,
      "erfolgswurf_rot": "nicht ausreichend",
      "auswirkungen": "Teilerfolg – eine knappe Beschreibung der operativen Auswirkungen gemäß Zusammenfassung des Spielleiters."
    }
  }
}

Ersetze alle erklärenden Platzhaltertexte (z.B. „eine kurze Zusammenfassung der Argumentation …“ oder „eine knappe Beschreibung …“)
durch konkrete, knappe Sätze, die zum Transkript passen.
Die numerischen Werte (S/M/L, Zahlen in Klammern, Tabellensumme, Erfolgswürfe) bleiben unverändert.

Antworte NUR mit diesem JSON, ohne Kommentar, ohne Markdown.

Transkript:
"""

    payload = {
        "model": MODEL_SUMMARY,
        "messages": [{"role": "user", "content": template + transcript}],
    }

    headers = {"Authorization": f"Bearer {API_KEY}", "Content-Type": "application/json"}
    resp = requests.post(BASE_URL, headers=headers, json=payload)
    resp.raise_for_status()

    result_raw = resp.json()["choices"][0]["message"]["content"]

    try:
        result_json = json.loads(result_raw)
    except json.JSONDecodeError:
        cleaned = result_raw.strip().strip("```json").strip("```").strip()
        result_json = json.loads(cleaned)

    # Statische Feldhinweise anhängen
    feldhinweise = {
        "intel_blau": {
            "beschreibung": "Bewertung des Informationsvorteils von Blau (z.B. ob ein Zielwert wie 12 erreicht wurde).",
            "moegliche_inhalte": "Hinweis, ob Blau ausreichend gewürfelt hat und welche Konsequenz das für die Sichtbarkeit von Rot hat."
        },
        "intel_rot": {
            "beschreibung": "Beschreibung des Informationsvorteils von Rot inklusive Wurfergebnis und Zielrichtung.",
            "moegliche_inhalte": "Kein Intelvorteil (z.B. bei Wurf 1) oder ein Vorteil mit Bonus gegen bestimmte Ziele (z.B. +2 gegen Personal)."
        },
        "ressourcen": {
            "beschreibung": "Einschätzung, wie gut Ressourcen (Personal, Material, Zeit, Infrastruktur) die Aktion unterstützen.",
            "moegliche_inhalte": "Format wie 'S (1) …', 'M (2) …', 'L (3) …' mit kurzer Begründung der Einstufung."
        },
        "komplexitaet": {
            "beschreibung": "Bewertung der Komplexität der Aktion bzw. des Plans.",
            "moegliche_inhalte": "Kurze Begründung, warum die Aktion eher einfach (S), mittel (M) oder hoch komplex (L) ist."
        },
        "verteidigung": {
            "beschreibung": "Einschätzung der gegnerischen Verteidigungsmöglichkeiten und -stärke.",
            "moegliche_inhalte": "Begründung, warum die Verteidigung als stark, mittel oder schwach bewertet wurde."
        },
        "angriffszeitfenster": {
            "beschreibung": "Bewertung, wie günstig das Zeitfenster für den Angriff ist.",
            "moegliche_inhalte": "Hinweise auf Timing, Überraschungseffekt, Verfügbarkeit von Kräften im relevanten Zeitraum."
        },
        "tabellensumme": {
            "beschreibung": "Numerische Summe der Einzelbewertungen (Ressourcen, Komplexität etc.).",
            "moegliche_inhalte": "Ganzzahl, z.B. 8, die direkt aus der zugrundeliegenden Bewertungstabelle stammt."
        },
        "erfolgswurf_rot": {
            "beschreibung": "Ergebnis des Erfolgswurfs von Rot für die Aktion in dieser Runde.",
            "moegliche_inhalte": "Text wie 'erfolgreich' oder 'nicht ausreichend', abhängig vom Wurf im Verhältnis zur Tabellensumme."
        },
        "auswirkungen": {
            "beschreibung": "Kurzbeschreibung der operativen und narrativen Folgen (Ergebnis im Szenario).",
            "moegliche_inhalte": "Z.B. 'Voller Erfolg' oder 'Teilerfolg' mit 1–2 Sätzen zur Wirkung im Szenario, basierend auf der Spielleiter-Zusammenfassung."
        }
    }

    result_json["feldhinweise"] = feldhinweise

    with open(OUTPUT_ONEPAGER_JSON_PATH, "w", encoding="utf-8") as f:
        json.dump(result_json, f, ensure_ascii=False, indent=2)

    return result_json


def main():
    print("Starte Transkription...")
    transcript = transcribe_audio()
    print("Transkript gespeichert in:", OUTPUT_TRANSCRIPT_PATH)

    print("Erstelle One-Pager (JSON)...")
    one_pager = create_one_pager_json(transcript)
    print("One-Pager gespeichert als:", OUTPUT_ONEPAGER_JSON_PATH)

    print("\nVorschau JSON:\n")
    print(json.dumps(one_pager, indent=2, ensure_ascii=False))


if __name__ == "__main__":
    main()


Starte Transkription...
Transkript gespeichert in: transkript_mit_sprechern.txt
Erstelle One-Pager (JSON)...
One-Pager gespeichert als: one_pager.json

Vorschau JSON:

{
  "runde1": {
    "phase1": {
      "intel_blau": "Blau hat die 12 nicht erreicht und daher ist der Intelvorteil von Rot geheim.",
      "intel_rot": "Rot hat keinen Intelvorteil, da 1 gewürfelt."
    },
    "phase2": {
      "ressourcen": "S (1) Der Angriff erfordert minimale Ressourcen, da ein allgemein befähigter Innentäter genutzt wird und nur eine kurze Beobachtung des Opfers notwendig ist.",
      "komplexitaet": "M (2) Der Angriff ist aufgrund unvorhersehbarer Variablen wie plötzlichem Zurückkehren des Opfers und der Notwendigkeit zur präzisen Beobachtung mäßig komplex.",
      "verteidigung": "L (3) Die Verteidigung ist gut, da die Mehrheit der Mitarbeiter ihre Geräte schützt und sensibilisiert ist.",
      "angriffszeitfenster": "M (2) Das Zeitfenster umfasst mehrere Tage von der Durchführung bis zur Wirkung, 